In [114]:
!curl -L https://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data -o letter-recognition.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  695k  100  695k    0     0   951k      0 --:--:-- --:--:-- --:--:--  951k


In [115]:
# importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from math import exp,log
from collections import defaultdict 
from sklearn.metrics.pairwise import chi2_kernel

In [116]:
df = pd.read_csv('letter-recognition.csv')

# feature names were like - 'letter ' changed it to 'letter'
df.columns = ['letter', 'xbox', 'ybox', 'width', 'height', 'onpix', 'xbar',
       'ybar', 'x2bar', 'y2bar', 'xybar', 'x2ybar', 'xy2bar', 'xedge',
       'xedgey', 'yedge', 'yedgex']
print(df.columns)

Index(['letter', 'xbox', 'ybox', 'width', 'height', 'onpix', 'xbar', 'ybar',
       'x2bar', 'y2bar', 'xybar', 'x2ybar', 'xy2bar', 'xedge', 'xedgey',
       'yedge', 'yedgex'],
      dtype='object')


In [117]:
print(f"Number of rows = {df.shape[0]}, cols = {df.shape[1]}")
df.head()

Number of rows = 19999, cols = 17


,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
1,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
2,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
3,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
4,S,4,11,5,8,3,8,8,6,9,5,6,6,0,8,9,7


In [118]:
# number of different classes
classes = df['letter'].unique()
print(f'Number of different labels in dataset = {len(classes)}')
classes.sort()
for cls in classes:
    rows = len(df[df['letter'] == cls])
    # print(f'Number of data points in class {cls} = {rows}')

Number of different labels in dataset = 26


In [119]:
def predict(z,gt):
  x = z.loc[:, z.columns != 'letter']
  y = z.iloc[0]['letter']

  fx = gt.decision_function(x)
  # print(f"Decision = {fx} prediction = {gt.predict(x)} actual = {y}")
  # w_norm = np.linalg.norm(gt.coef_)  
  return fx

In [120]:
# returns the loss value calculated using model "gt" for datapoint "z"
def loss(z,gt):
  y = z.iloc[0]['letter']
  # doubtful
  fx = (predict(z,gt))[0]
  # print(fx,y)
  if fx*y >= 1:
    return 0
  else:
    return 1-fx*y

In [121]:
'''
  zs = numerator
  zi = denominator
  gt = trained model of svm, using which loss is calculated
'''
def calculate_P(zs,zi,gt):
  P = exp(-loss(zs,gt))/exp(-loss(zi,gt))
  return P

In [122]:
def acceptance_prob(zs,zst,gt):
  ys = zs.iloc[0]['letter']
  yt = zst.iloc[0]['letter']

  fs = predict(zs,gt)
  ft = predict(zst,gt)

  return exp(-fs[0]*ys)/exp(-ft[0]*yt)

In [123]:
def hellinger(X1, X2):
  return np.sqrt(np.dot(X1,X2.T))

In [124]:
# Preparing the dataset

kernels = ['hellinger','linear','chi2_kernel','rbf', 'poly']

for kernel in kernels:
  print(f"Running for Kernel {kernel} ...")
  correct = 0
  total = 0
  for alp in classes:

    # binary labeling of the dataset
    df1 = df.copy()

    # one alphabet will be treated as positive sample and others as negative sample
    alphabet = alp  
    print(f"Running for {alphabet} ...")
    df1.loc[(df1.letter != alphabet),'letter'] = -1
    df1.loc[(df1.letter == alphabet),'letter'] = 1

    # we will make number of positve samples = number of negative samples otherwise
    # otherwise the dataset will have huge number of negative samples which will slow
    # down the sampling process
    pos = df1[df1['letter']==1].sample(frac=1)
    # we need roughly only 1/25 of total negative samples 
    neg = df1[df1['letter']==-1].sample(frac=1/25)

    # print(f"Final pos = {len(pos)} and neg = {len(neg)}")
    df_final = pos.append(neg,ignore_index=True)
    # print(f"Length of final dataset {len(df_final)}")
    df_final.head()

    # train test split
    D_train, D_test = train_test_split(df_final, test_size=0.3)

    # inputs of the model
    N = 500               # size of dataset generated in each iteration(= Dt)
    n2 = 10               # Threhsold for increasing acceptance probability of zi
    q = 1.3               # Multiplier with which acceptance probability is increased
    T = 3               # Number of different models to be generated


    D_train_features = (D_train.loc[:, D_train.columns != 'letter'])
    D_train_target = D_train.loc[:, ['letter']].astype('int').values.ravel()

    # we will extract N data points randomly
    Dt = D_train.sample(n=N)
    Dt_features = (Dt.loc[:, Dt.columns != 'letter'])
    Dt_target = Dt.loc[:, ['letter']].astype('int').values.ravel()

    g = []

    for i in range(0,T+1):
      if kernel == 'chi2_kernel':
        g.append(SVC(kernel=chi2_kernel))
      elif kernel == 'hellinger':
        g.append(SVC(kernel=hellinger))
      else:
        g.append(SVC(kernel=kernel))
    e = [0]*(T+1)
    alpha = [1]*(T+1)

    # g[0] the first trained SVM model, since we are doing training multiple times 
    g[0].fit(Dt_features, Dt_target)

    zi = D_train.sample(n=1)
    # "t" = count of number of SVM models generated, a total of 'T' SVM models are generated
    t = 1 
    while t <= T:
      # print(f"Training model : {t} ...")
      '''
      n1 = number of times the previous sample zi was rejected
      in case zi was rejected a lot of times(i.e > n2),
      we will increase its acceptance probability by multiplying with Q which is greater than 1
      '''
      n1 = 0
      # Dt = new sampled data on which svm will be trained(= g[t]) in the next iteration
      Dt = []
      # "i" is used to count the current size of "Dt", we will sample "N" datapoints in new dataset Dt
      i = 1
      p=0
      n=0
      while i <= N:
        
        # draw one sample randomly = z_star(zs)
        zs = D_train.sample(n=1)
        # calculate acceptance probability using previous model g[t-1]
        pt = min(1,calculate_P(zs,zi,g[t-1]))

        if n1 > n2:
          pt = min(1,q*pt)
          # Dt.append(zs)
          # zi = zs
          # i = i+1
          # n1 = 0
        
        # ys corresponds to Y of zs
        ys = zs.iloc[0]['letter']
        yi = zi.iloc[0]['letter']
        
        if pt == 1 and ys*yi==1:
          pt = acceptance_prob(zs,zi,g[t-1])
        
        # print(f"Label : {ys} Prob : {pt}")
        if np.random.random() < pt :
          # accept zs 
          if ys==1:
            p+=1
          else:
            n+=1
          Dt.append(zs)
          zi = zs
          i = i+1
          n1 = 0

        # count number of times zs is rejected(= n1)
        else :
          n1 = n1+1
      
      # train the model to be used in next iteration g[t]
      # print(f"Length = {len(Dt)} positive = {p} negative = {n}")
      Dt = pd.concat(Dt)

      Dt_features = (Dt.loc[:, Dt.columns != 'letter'])
      Dt_target = Dt.loc[:, ['letter']].astype('int').values.ravel()

      g[t].fit(Dt_features, Dt_target)

      D_train_prediction = g[t].predict(D_train_features)
      acc = accuracy_score(y_true = D_train_target, y_pred = D_train_prediction)
      # error or misclassification rate for model g[t]
      e[t] = 1 - acc
      alpha[t] = 0.5 * log((1-e[t])/e[t])
      zi = zs
      if alpha[t] < 0 :
        t = t-1
      t = t+1
    D_test_features = (D_test.loc[:, D_test.columns != 'letter'])
    D_test_target = D_test.loc[:, ['letter']].astype('int').values.ravel()

    D_test_predicted = []
    for t in range(0,T+1):
      D_test_predicted.append(g[t].decision_function(D_test_features))

    for i in range(len(D_test_target)):

      y_target = D_test_target[i]
      sum = 0
      for j in range(1,T+1):
        sum += alpha[j]*D_test_predicted[j][i]
      y_predicted = np.sign(sum)

      total +=1 
      if y_predicted == y_target:
        correct+=1

  print(f"Accuracy for kernel {kernel} = {correct/total}")

Running for Kernel hellinger ...
Running for A ...
Running for B ...
Running for C ...
Running for D ...
Running for E ...
Running for F ...
Running for G ...
Running for H ...
Running for I ...
Running for J ...
Running for K ...
Running for L ...
Running for M ...
Running for N ...
Running for O ...
Running for P ...
Running for Q ...
Running for R ...
Running for S ...
Running for T ...
Running for U ...
Running for V ...
Running for W ...
Running for X ...
Running for Y ...
Running for Z ...
Accuracy for kernel hellinger = 0.8910256410256411
Running for Kernel linear ...
Running for A ...
Running for B ...
Running for C ...
Running for D ...
Running for E ...
Running for F ...
Running for G ...
Running for H ...
Running for I ...
Running for J ...
Running for K ...
Running for L ...
Running for M ...
Running for N ...
Running for O ...
Running for P ...
Running for Q ...
Running for R ...
Running for S ...
Running for T ...
Running for U ...
Running for V ...
Running for W ...
Runn